In [1]:
ls

data/                 preprocessor.ipynb      testing_live.ipynb
model_training.ipynb  scaler_t.ipynb          user_parameters.json
output/               selected_features.json


In [2]:
!pip install openpyxl==3.1.0


[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
from pathlib import Path

# === INPUT YOUR FILE PATH HERE ===
file_path = 'data/concat_ESData.xlsx'  # Replace with your actual file path

# Convert to Path object
file_path = Path(file_path)

# Read the Excel file
df = pd.read_excel(file_path)

# Drop duplicate rows
df_no_duplicates = df.drop_duplicates()

# Create output file path in the same directory
output_filename = file_path.stem + '_no_duplicates.csv'
output_path = file_path.parent / output_filename

# Save to CSV at the same location
df_no_duplicates.to_csv(output_path, index=False)

print(f"✅ Duplicate rows removed. File saved at:\n{output_path}")


✅ Duplicate rows removed. File saved at:
data/concat_ESData_no_duplicates.csv


In [11]:
df_no_duplicates.shape

(104179, 21)

In [5]:
import pandas as pd
from pathlib import Path

# === INPUT your _no_duplicates file path ===
file_path = Path('data/concat_ESData_no_duplicates.csv')  # Replace with actual path

# Read the CSV file
df = pd.read_csv(file_path)

# Columns to keep (plus 'Date' if needed for reference)
continuous_cols = [
    'Open', 'High', 'Low', 'Close', 
    'RSI', 'MACD', 'HABodyRangeRatio', 'MyWAZLTTrend'
]

# If you want to keep 'Date' as the first column, uncomment this:
# if 'Date' in df.columns:
#     continuous_cols = ['Date'] + continuous_cols

# Filter the DataFrame
df_filtered = df[continuous_cols]

# Save to new CSV with '_for_scaler' in the name
output_filename = file_path.stem + '_for_scaler.csv'
output_path = file_path.parent / output_filename

df_filtered.to_csv(output_path, index=False)

print(f"✅ Scaler data saved at:\n{output_path}")


✅ Scaler data saved at:
data/concat_ESData_no_duplicates_for_scaler.csv


In [6]:
import pandas as pd
from pathlib import Path

# === Path to your _for_scaler.csv file ===
file_path = Path('data/concat_ESData_no_duplicates_for_scaler.csv')  # Replace with actual file

# Read the CSV
df = pd.read_csv(file_path)

# Print min and max for each column
print("📊 Min and Max values for each column:\n")
for col in df.columns:
    min_val = df[col].min()
    max_val = df[col].max()
    print(f"{col}: min = {min_val}, max = {max_val}")


📊 Min and Max values for each column:

Open: min = 4919.0, max = 6328.0
High: min = 4922.75, max = 6328.25
Low: min = 4918.75, max = 6327.75
Close: min = 4919.0, max = 6328.25
RSI: min = 0.24, max = 99.7
MACD: min = -24.83, max = 32.32
HABodyRangeRatio: min = 0.0, max = 1.0
MyWAZLTTrend: min = -1.0, max = 1.0


In [7]:
import pandas as pd
from pathlib import Path

# === Path to your _for_scaler.csv file ===
file_path = Path('data/concat_ESData_no_duplicates_for_scaler.csv')  # Replace with actual path

# Read CSV
df = pd.read_csv(file_path)

# Calculate min and max
min_vals = df.min()
max_vals = df.max()

# Create top (first) row with 20% less of min values and RSI = 0
top_row = {
    'Open': min_vals['Open'] * 0.8,
    'High': min_vals['High'] * 0.8,
    'Low': min_vals['Low'] * 0.8,
    'Close': min_vals['Close'] * 0.8,
    'RSI': 0,
    'MACD': min_vals['MACD'],
    'HABodyRangeRatio': min_vals['HABodyRangeRatio'],
    'MyWAZLTTrend': min_vals['MyWAZLTTrend']
}

# Create bottom (last) row with 20% more of max values and RSI = 100
bottom_row = {
    'Open': max_vals['Open'] * 1.2,
    'High': max_vals['High'] * 1.2,
    'Low': max_vals['Low'] * 1.2,
    'Close': max_vals['Close'] * 1.2,
    'RSI': 100,
    'MACD': max_vals['MACD'],
    'HABodyRangeRatio': max_vals['HABodyRangeRatio'],
    'MyWAZLTTrend': max_vals['MyWAZLTTrend']
}

# Insert top row at the beginning
df = pd.concat([pd.DataFrame([top_row]), df], ignore_index=True)

# Append bottom row at the end
df = pd.concat([df, pd.DataFrame([bottom_row])], ignore_index=True)

# Save new CSV
output_path = file_path.parent / (file_path.stem + '_updated_values.csv')
df.to_csv(output_path, index=False)

print(f"✅ New CSV with top/bottom rows saved at:\n{output_path}")


✅ New CSV with top/bottom rows saved at:
data/concat_ESData_no_duplicates_for_scaler_updated_values.csv


In [8]:
import pandas as pd
from pathlib import Path

file_path = Path('data/concat_ESData_no_duplicates_for_scaler_updated_values.csv')  


df = pd.read_csv(file_path)


print("📊 Min and Max values for each column:\n")
for col in df.columns:
    min_val = df[col].min()
    max_val = df[col].max()
    print(f"{col}: min = {min_val}, max = {max_val}")


📊 Min and Max values for each column:

Open: min = 3935.2, max = 7593.599999999999
High: min = 3938.2, max = 7593.9
Low: min = 3935.0, max = 7593.299999999999
Close: min = 3935.2, max = 7593.9
RSI: min = 0.0, max = 100.0
MACD: min = -24.83, max = 32.32
HABodyRangeRatio: min = 0.0, max = 1.0
MyWAZLTTrend: min = -1.0, max = 1.0


In [10]:
import pandas as pd
import joblib
from pathlib import Path
from datetime import datetime
from sklearn.preprocessing import StandardScaler

# === File path to your CSV ===
file_path = Path('data/concat_ESData_no_duplicates_for_scaler_updated_values.csv')

# === Directory where scaler will be saved ===
scaler_output_path = Path('data/ES_scaler.pkl')  # Adjust as needed

# === Required base columns ===
base_cols = [
    'Open', 'High', 'Low', 'Close', 
    'RSI', 'MACD', 'HABodyRangeRatio', 'MyWAZLTTrend'
]

# === Load the CSV ===
try:
    df = pd.read_csv(file_path)
except Exception as e:
    raise RuntimeError(f"❌ Failed to read CSV: {file_path}\nError: {e}")

# === Check all base columns exist ===
missing = [col for col in base_cols if col not in df.columns]
if missing:
    raise ValueError(f"❌ Missing required columns in CSV: {missing}")

# === Create *_lag1 columns by copying base values ===
for col in base_cols:
    df[col + '_lag1'] = df[col]

# === Final list of continuous columns ===
continuous_cols = base_cols + [col + '_lag1' for col in base_cols]

# === Create and save StandardScaler ===
def create_and_save_scaler(df, columns, output_path):
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Creating scaler for continuous columns...")

    scaler = StandardScaler()
    scaler.fit(df[columns])

    joblib.dump(scaler, output_path)

    print("✅ Scaler saved to:", output_path)
    print("📊 Column statistics:")
    for col in columns:
        print(f"{col}: mean={df[col].mean():.4f}, std={df[col].std():.4f}")

    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Done.")

# === Run the scaler function ===
create_and_save_scaler(df, continuous_cols, scaler_output_path)


2025-08-02 10:27:25: Creating scaler for continuous columns...
✅ Scaler saved to: data/ES_scaler.pkl
📊 Column statistics:
Open: mean=5861.8215, std=320.1473
High: mean=5862.4344, std=319.7642
Low: mean=5861.2024, std=320.5156
Close: mean=5861.8185, std=320.1470
RSI: mean=50.7307, std=21.8508
MACD: mean=0.0208, std=1.4163
HABodyRangeRatio: mean=0.3187, std=0.1965
MyWAZLTTrend: mean=0.0258, std=0.6190
Open_lag1: mean=5861.8215, std=320.1473
High_lag1: mean=5862.4344, std=319.7642
Low_lag1: mean=5861.2024, std=320.5156
Close_lag1: mean=5861.8185, std=320.1470
RSI_lag1: mean=50.7307, std=21.8508
MACD_lag1: mean=0.0208, std=1.4163
HABodyRangeRatio_lag1: mean=0.3187, std=0.1965
MyWAZLTTrend_lag1: mean=0.0258, std=0.6190
2025-08-02 10:27:25: Done.
